In [3]:
import random
import pandas as pd
from datetime import datetime

# Temporary workflow
# (1) Download csv with `heroku run make exportdb && make importdb`
# (2) save links to ~/Documents/links.txt in onetab format
# (3) run this script (or otherwise modify the CSV as desired)
# (4) check date cell
# (5) Upload back up with `make csv_to_s3 && heroku run make importdb`
links = pd.read_csv('data/export.csv')
links

/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_20118/1281081789.py:11: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  links = pd.read_csv('data/export.csv')


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93637,155809,https://warontherocks.com/2022/08/cooperation-...,Cooperation Can Make the NATO Lake a Reality,\N,\N,2022-08-29 15:25:50.094985+00,2022-08-29 15:25:50.094985+00,NaN,\N,WarOnRocks,1.0,0.0,\N
93638,155810,https://forum.effectivealtruism.org/posts/zvNw...,Questioning the Foundations of EA,\N,forum.effectivealtruism.org,2022-08-29 15:25:50.096225+00,2022-08-29 17:22:16.882846+00,-1.0,\N,EAForum,50.0,0.0,\N
93639,155811,https://fivethirtyeight.com/features/trumps-en...,Trump’s Endorsees Have Started Losing More. Bu...,\N,\N,2022-08-29 15:25:50.097303+00,2022-08-29 15:25:50.097303+00,NaN,\N,538,30.0,0.0,\N
93640,155812,https://aligned.substack.com/p/ai-assisted-hum...,Why I’m excited about AI-assisted human feedback,\N,\N,2022-08-29 15:25:50.09836+00,2022-08-29 15:25:50.09836+00,NaN,\N,Leike,89.0,0.0,\N


In [4]:
links['added'].max()

'2022-08-29 16:51:16.739352+00'

In [5]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]['id']

1299       1306
1300       1307
1301       1308
1302       1309
1303       1310
          ...  
93281    155445
93282    155446
93283    155447
93284    155448
93641    155814
Name: id, Length: 9907, dtype: int64

In [6]:
links[links['url'].duplicated()]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
93617,155789,https://www.centreforeffectivealtruism.org/blo...,CEA’s 2017 review and 2018 plans,\N,\N,2022-08-29 15:25:50.071809+00,2022-08-29 15:25:50.071809+00,NaN,\N,CEA,1.0,0.0,\N


In [7]:
links[links['url'].duplicated()]['id']

93617    155789
Name: id, dtype: int64

In [8]:
link_file = open('/Users/peterhurford/Documents/links.txt', 'r')
new_links = link_file.readlines()

In [9]:
def new_entry(links, url, title):
    return {'id': links['id'].max() + 1,
            'url': str(url).strip().replace(',', ''),
            'title': str(title).strip().replace(',', ''),
            'summary': '\\N',
            'domain': '\\N',
            'added': str(datetime.now()),
            'modified': str(datetime.now()),
            'liked': '\\N',
            'category': '\\N',
            'aggregator': 'Custom',
            'seed': random.randint(1, 100),
            'tweet': 0}

for i, new_link in enumerate(new_links):
    if i % 10 == 0 or i == len(new_links) - 1:
        print('...{}/{}'.format(i + 1, len(new_links)))
    if ' | ' in new_link:
        new_link = new_link.replace('    - ', '').replace('\n', '')
        entry = new_entry(links, new_link.split(' | ')[0], new_link.split(' | ')[1])
        links = links.append(entry, ignore_index=True)

links

...1/177
...11/177


/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_20118/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_20118/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_20118/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_20118/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

...21/177
...31/177


/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_20118/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_20118/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_20118/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_20118/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

...41/177
...51/177


...

/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_20118/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_20118/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_20118/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_20118/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

...141/177
...151/177


/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_20118/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_20118/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_20118/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_20118/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

...161/177
...171/177
...177/177


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93814,155987,https://www.nytimes.com/2022/08/24/technology/...,We Need to Talk About How Good A.I. Is Getting...,\N,\N,2022-08-29 18:31:58.239875,2022-08-29 18:31:58.239886,\N,\N,Custom,27.0,0.0,NaN
93815,155988,https://psyarxiv.com/tnf4e/,PsyArXiv Preprints,\N,\N,2022-08-29 18:31:58.251564,2022-08-29 18:31:58.251574,\N,\N,Custom,67.0,0.0,NaN
93816,155989,https://www.lesswrong.com/posts/JKgGvJCzNoBQss...,Beliefs and Disagreements about Automating Ali...,\N,\N,2022-08-29 18:31:58.263102,2022-08-29 18:31:58.263114,\N,\N,Custom,39.0,0.0,NaN
93817,155990,https://www.lesswrong.com/posts/gdyfJE3noRFSs3...,Resources I send to AI researchers about AI sa...,\N,\N,2022-08-29 18:31:58.273785,2022-08-29 18:31:58.273798,\N,\N,Custom,38.0,0.0,NaN


In [10]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
1299,1306,http://blog.beeminder.com/trust/,Trusting Your Divided Self | Beeminder Blog,\N,blog.beeminder.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,10.0,0.0,0
1300,1307,http://powlyglot.com/fi3m-review-part-1-focus/,"""What I Learned from Fluent in 3 Months\, Part...",\N,powlyglot.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,9.0,0.0,0
1301,1308,http://effective-altruism.com/ea/u8/ea_intervi...,"""EA Interview Series\, February 2016: John Sal...",\N,effective-altruism.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,effectivealtruism,Custom,85.0,0.0,0
1302,1309,https://fivethirtyeight.com/features/most-2020...,Most 2020 Candidates Have Something In Common:...,\N,fivethirtyeight.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,1.0,politicalscience,Custom,82.0,0.0,0
1303,1310,http://www.smartpassiveincome.com/backlinking-...,THE Backlinking Strategy That Works – 2014 and...,\N,www.smartpassiveincome.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,entrepreneurship,Custom,59.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93814,155987,https://www.nytimes.com/2022/08/24/technology/...,We Need to Talk About How Good A.I. Is Getting...,\N,\N,2022-08-29 18:31:58.239875,2022-08-29 18:31:58.239886,\N,\N,Custom,27.0,0.0,NaN
93815,155988,https://psyarxiv.com/tnf4e/,PsyArXiv Preprints,\N,\N,2022-08-29 18:31:58.251564,2022-08-29 18:31:58.251574,\N,\N,Custom,67.0,0.0,NaN
93816,155989,https://www.lesswrong.com/posts/JKgGvJCzNoBQss...,Beliefs and Disagreements about Automating Ali...,\N,\N,2022-08-29 18:31:58.263102,2022-08-29 18:31:58.263114,\N,\N,Custom,39.0,0.0,NaN
93817,155990,https://www.lesswrong.com/posts/gdyfJE3noRFSs3...,Resources I send to AI researchers about AI sa...,\N,\N,2022-08-29 18:31:58.273785,2022-08-29 18:31:58.273798,\N,\N,Custom,38.0,0.0,NaN


In [11]:
def clean(txt):
    if not isinstance(txt, str):
        return txt
    txt = txt.replace('&nbsp;', '')
    txt = txt.replace('&ldquo;', '"')
    txt = txt.replace('&rdquo;', '"')
    txt = txt.replace('&lsquo;', '\'')
    txt = txt.replace('&rsquo;', '\'')
    txt = txt.replace('&mdash;', '-')
    txt = txt.replace('&ndash;', '-')
    for i in range(10):
        txt = txt.replace('"""', '"')
        txt = txt.replace('\'\'\'', '\'')
    txt = txt.replace('n"t', 'n\'t')
    txt = txt.replace('n""t', 'n\'t')
    return txt

links = links[links['id'].notnull()]   # Drop empty column
links['id'] = links['id'].astype(int)  # Fix float ID issue
links['seed'] = links['seed'].astype(int)
links['tweet'] = links['tweet'].apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int)
links['summary'] = links['summary'].apply(clean)
links['title'] = links['title'].apply(clean)
links = links.sort_values('id')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17,0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59,0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56,0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29,0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93814,155987,https://www.nytimes.com/2022/08/24/technology/...,We Need to Talk About How Good A.I. Is Getting...,\N,\N,2022-08-29 18:31:58.239875,2022-08-29 18:31:58.239886,\N,\N,Custom,27,0,NaN
93815,155988,https://psyarxiv.com/tnf4e/,PsyArXiv Preprints,\N,\N,2022-08-29 18:31:58.251564,2022-08-29 18:31:58.251574,\N,\N,Custom,67,0,NaN
93816,155989,https://www.lesswrong.com/posts/JKgGvJCzNoBQss...,Beliefs and Disagreements about Automating Ali...,\N,\N,2022-08-29 18:31:58.263102,2022-08-29 18:31:58.263114,\N,\N,Custom,39,0,NaN
93817,155990,https://www.lesswrong.com/posts/gdyfJE3noRFSs3...,Resources I send to AI researchers about AI sa...,\N,\N,2022-08-29 18:31:58.273785,2022-08-29 18:31:58.273798,\N,\N,Custom,38,0,NaN


In [12]:
links.to_csv('data/export.csv', index=False)